<a href="https://colab.research.google.com/github/H-y-hoon/2021_School_Seminar/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import mglearn
from matplotlib import font_manager, rc 

font_name=font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name() 
rc('font', family=font_name) 

In [ ]:
df_train = pd.read_csv("C:/users/yhg31/OneDrive/바탕 화면/데이터파일/2021 학술제 데이터/Subject_B.csv")
df_test = pd.read_csv("C:/users/yhg31/OneDrive/바탕 화면/데이터파일/2021 학술제 데이터/Subject_D_testset.csv")

In [ ]:
#용량 줄이는 코드
import numpy as np

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    return df
df_train = reduce_mem_usage(df_train)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Columns: 113 entries, Class to AF4 beta m
dtypes: float16(112), int8(1)
memory usage: 527.5 KB


In [ ]:
df_train

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m
0,2,3578.00,2068.00,1.03,1.74,0.88,1.28,3.04,2.73,3568.00,...,1.49,2.04,3574.00,2064.00,1.03,2.87,0.43,1.62,2.07,2.24
1,2,3578.00,2066.00,0.57,1.71,1.01,1.90,2.75,3.07,3570.00,...,1.71,2.18,3574.00,2066.00,0.68,2.22,0.62,1.93,1.93,2.34
2,2,3576.00,2066.00,0.53,1.58,0.20,1.04,2.69,2.98,3570.00,...,1.55,2.11,3574.00,2064.00,0.61,2.00,0.35,1.63,2.02,2.51
3,2,3578.00,2066.00,0.55,1.32,0.37,1.04,2.55,2.91,3568.00,...,1.78,2.22,3574.00,2064.00,0.71,1.41,0.73,1.09,2.11,2.42
4,2,3578.00,2066.00,0.25,1.17,0.26,1.17,2.68,2.87,3568.00,...,2.11,2.16,3574.00,2066.00,0.17,1.15,0.54,1.74,1.87,2.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0,3574.00,2066.00,1.01,2.40,0.64,2.96,2.07,2.41,3566.00,...,1.59,2.00,3572.00,2064.00,0.96,1.84,0.72,1.94,1.58,2.38
2396,0,3574.00,2064.00,1.45,2.68,0.22,1.65,1.92,2.50,3566.00,...,1.74,1.98,3572.00,2064.00,1.52,1.55,0.91,1.40,1.43,2.29
2397,0,3574.00,2064.00,1.80,2.19,0.12,1.37,1.89,2.76,3566.00,...,1.72,2.02,3572.00,2064.00,2.08,2.28,0.79,1.91,1.44,2.29
2398,0,3574.00,2064.00,1.34,1.93,0.80,1.33,2.25,2.65,3566.00,...,2.03,2.03,3572.00,2064.00,1.82,2.82,0.49,1.88,1.54,2.37


In [ ]:
df_train["Class"].value_counts()

2    800
1    800
0    800
Name: Class, dtype: int64

In [ ]:
col_list=df_train.columns[1:]
col_list1=[]
for i in range(len(col_list)):
    col_list1.append(col_list[i].split(" ")[0])
col_list1
set(col_list1)

{'AF3',
 'AF4',
 'F3',
 'F4',
 'F7',
 'F8',
 'FC5',
 'FC6',
 'O1',
 'O2',
 'P7',
 'P8',
 'T7',
 'T8'}

In [ ]:
df_train[df_train.duplicated()]

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Columns: 113 entries, Class to AF4 beta m
dtypes: float16(112), int8(1)
memory usage: 527.5 KB


In [ ]:
df_train.describe()

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m
count,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,...,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00,2400.00
mean,1.00,inf,inf,0.92,2.12,0.74,1.80,2.20,2.62,inf,...,1.64,2.04,inf,inf,0.91,2.02,0.72,1.76,1.61,2.26
std,0.82,4.07,2.43,0.42,0.56,0.34,0.55,0.57,0.36,2.43,...,0.41,0.32,3.08,1.86,0.42,0.53,0.33,0.54,0.44,0.28
min,0.00,3556.00,2054.00,0.04,0.48,0.07,0.48,0.63,1.31,3560.00,...,0.65,1.15,3558.00,2058.00,0.05,0.50,0.04,0.49,0.56,1.35
25%,0.00,3572.00,2064.00,0.63,1.74,0.49,1.42,1.83,2.37,3566.00,...,1.34,1.83,3570.00,2062.00,0.60,1.65,0.47,1.39,1.30,2.07
50%,1.00,3574.00,2066.00,0.89,2.10,0.70,1.75,2.19,2.62,3568.00,...,1.61,2.02,3572.00,2064.00,0.86,2.00,0.69,1.71,1.55,2.25
75%,2.00,3576.00,2066.00,1.17,2.45,0.93,2.15,2.60,2.87,3570.00,...,1.89,2.24,3574.00,2066.00,1.17,2.33,0.93,2.07,1.87,2.44
max,2.00,3580.00,2068.00,2.95,4.25,2.34,4.33,3.74,3.87,3574.00,...,2.91,3.21,3578.00,2070.00,2.57,4.56,3.26,4.93,3.32,3.71


In [ ]:
def make_data(x, y, window_size):
    x_list=[]
    y_list=[]
    
    for i in range(len(x)-window_size):
        x_list.append(x[i:i+window_size])
        y_list.append(y[i+window_size])

    return np.array(x_list), np.array(y_list)

In [ ]:
X = df_train.iloc[:, 1:]
Y  = df_train.iloc[:, 0]

print(X.shape, Y.shape)

(2400, 112) (2400,)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)
x_scaled = pd.DataFrame(x_scaled, columns=X.columns)
x_scaled

C:\Users\yhg31\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)


,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m
0,1.38,1.49,0.24,-0.68,0.42,-0.95,1.48,0.29,-0.15,-0.06,...,-0.36,-0.00,0.80,0.07,0.27,1.60,-0.89,-0.27,1.05,-0.05
1,1.38,0.65,-0.84,-0.74,0.80,0.18,0.96,1.21,0.68,-0.06,...,0.18,0.45,0.80,1.15,-0.56,0.38,-0.32,0.30,0.75,0.29
2,0.88,0.65,-0.95,-0.96,-1.59,-1.40,0.86,0.97,0.68,-0.06,...,-0.20,0.23,0.80,0.07,-0.74,-0.03,-1.11,-0.24,0.94,0.88
3,1.38,0.65,-0.89,-1.43,-1.08,-1.39,0.61,0.78,-0.15,-0.06,...,0.34,0.56,0.80,0.07,-0.48,-1.15,0.03,-1.25,1.15,0.56
4,1.38,0.65,-1.60,-1.70,-1.42,-1.16,0.85,0.66,-0.15,-1.46,...,1.16,0.38,0.80,1.15,-1.79,-1.63,-0.55,-0.05,0.60,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0.39,0.65,0.21,0.49,-0.28,2.11,-0.23,-0.58,-0.99,-1.46,...,-0.12,-0.12,0.15,0.07,0.11,-0.34,-0.01,0.32,-0.05,0.44
2396,0.39,-0.18,1.25,0.99,-1.53,-0.28,-0.49,-0.33,-0.99,-1.46,...,0.24,-0.20,0.15,0.07,1.44,-0.88,0.57,-0.66,-0.41,0.11
2397,0.39,-0.18,2.08,0.12,-1.82,-0.79,-0.54,0.37,-0.99,-1.46,...,0.22,-0.05,0.15,0.07,2.79,0.50,0.19,0.27,-0.39,0.09
2398,0.39,-0.18,1.00,-0.34,0.18,-0.86,0.08,0.08,-0.99,-1.46,...,0.97,-0.04,0.15,0.07,2.17,1.52,-0.70,0.22,-0.16,0.40


In [ ]:
train, test = make_data(x_scaled, Y, 10)
print(train.shape, test.shape)

(2390, 10, 112) (2390,)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test  = train_test_split(train, test, test_size=0.3)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1673, 10, 112)
x_test shape: (717, 10, 112)
y_train shape: (1673,)
y_test shape: (717,)


In [ ]:
from sklearn.model_selection import train_test_split

x_train1, x_test1, y_train1, y_test1  = train_test_split(x_scaled, Y, test_size=0.3)
print('x_train1 shape:', x_train1.shape)
print('x_test1 shape:', x_test1.shape)
print('y_train1 shape:', y_train1.shape)
print('y_test1 shape:', y_test1.shape)

x_train1 shape: (1680, 112)
x_test1 shape: (720, 112)
y_train1 shape: (1680,)
y_test1 shape: (720,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(4))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               123392    
                                                                 
 dense_4 (Dense)             (None, 4)                 516       
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 123,913
Trainable params: 123,913
Non-trainable params: 0
_________________________________________________________________


In [ ]:
early_stop = EarlyStopping(monitor='accuracy', patience=1, verbose=1)

model.fit(x_train, y_train, epochs=100,
          batch_size=32, verbose=1, callbacks=[early_stop])

Epoch 1/100
53/53 [==============================] - 2s 13ms/step - loss: 0.4339 - accuracy: 0.4967
Epoch 2/100
53/53 [==============================] - 1s 13ms/step - loss: 0.0743 - accuracy: 0.6396
Epoch 3/100
53/53 [==============================] - 1s 13ms/step - loss: 0.0325 - accuracy: 0.6623
Epoch 4/100
53/53 [==============================] - 1s 13ms/step - loss: 0.0186 - accuracy: 0.6647
Epoch 5/100
53/53 [==============================] - 1s 14ms/step - loss: 0.0128 - accuracy: 0.6653
Epoch 6/100
53/53 [==============================] - 1s 13ms/step - loss: 0.0129 - accuracy: 0.6659
Epoch 7/100
53/53 [==============================] - 1s 13ms/step - loss: 0.0103 - accuracy: 0.6653
Epoch 00007: early stopping


In [ ]:
print("정확도 : %.2f"%(model.evaluate(x_test, y_test, batch_size=32)[1]))

23/23 [==============================] - 1s 6ms/step - loss: 0.0325 - accuracy: 0.6667
정확도 : 0.67


In [ ]:
lstm_train_pred = model.predict(x_train).astype(int)
lstm_test_pred = model.predict(x_test).astype(int)

In [ ]:
from sklearn.metrics import confusion_matrix
print("LSTM train\n", confusion_matrix(y_train, lstm_train_pred))
print()
print("LSTM test\n", confusion_matrix(y_test, lstm_test_pred))

LSTM train
 [[558   0   0]
 [306 251   0]
 [  0 327 231]]

LSTM test
 [[241   1   0]
 [116 127   0]
 [  1 159  72]]
